# Knowledge and Data 2020: Practical Assignment 1 
## Manipulate Propositional Logic and Simple Knowledge Graph Logic

- YOUR NAME: Michael Moor

- YOUR VUNetID: mmr497

*(If you do not provide your NAME and VUNetID we will not accept your submission.)*

### Learning objectives

At the end of this exercise you should be able to perform some simple manipulations of two different formal systems: 
1. You will be able to transform Propositional Logic statements from one syntactic form to semantically equivalent  alternative representation.
2. You will be able to evaluate a Propositional Logic statements given some validation of the propositional variables. 
3. You will be able to transform one Knowledge Graph into an equivalent one with a different Syntax 
4. You will be able to calculate simple entailment between two Knowledge Graph

### Practicalities

Follow this Notebook step-by-step. Some of the exercises are already 
fully filled in to give you some examples of how to do things. Others are
partially filled, and require you to fill in some gaps. And others, you 
will have to fully program yourself. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, save the Notebook and submit it 
as a Jupyter Notebook (i.e. with an ipynb extension). Please use as name of the 
Notebook your studentID+Assignment1.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

## Exercises related to Propositional Logic 

We will use a very simple, though crude and ugly, representation for Propositional Logic: Propositions are strings, and formulas are lists, starting with an operator as first element in the list, and the subformulas as second and (if appropriate) third argument. (Note that this is slightly different than the definition in the lecture, where Propositions are lists with one argument). These are formulas in Prefix Notation.

In the file, there is one formula per line. 

For simplicity, we will only consider formulas with any of the three variables P, Q and R, and binary operators ">> (if), & (and), | (or)" and "~ (not)". 

Here are some formulas as examples.

(~R & P) >> Q

~(((P | ~Q) >> R) >> (P & R)) 

~(P >> Q) | (R >> P) 

(P >> Q) | (Q >> ~P)


The first thing you will have to do is to open the file. The code below will read every line of the file into a list called *content*.

Do *not* forget to run the following cell, otherwise, the file will not be opened. import os 


In [82]:
import os 
from logic import *

In [83]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'PLknowledgebase.txt')
print(filename)
with open(filename) as file:
    content = file.readlines()
     
print(content)

/home/michael/Documents/GitHub/knowledge-data-vu/Assignments/Assignment_1/PLknowledgebase.txt
['(~R & P) >> Q\n', '~(((P | ~Q) >> R) >> (P & R)) \n', '~(P >> Q) | (R >> P) \n', '(P >> Q) | (Q >> ~P)']


In [84]:
P, Q, R = vars('P', 'Q', 'R')

for line in content:
    formula = eval(line.strip())
    print()
    print(formula)
    print(formula.is_tautology())
    print(type(formula))



(¬R ^ P) => Q
False
<class 'logic.Implies'>

¬(((P v ¬Q) => R) => (P ^ R))
False
<class 'logic.Not'>

¬(P => Q) v (R => P)
False
<class 'logic.Or'>

(P => Q) v (Q => ¬P)
True
<class 'logic.Or'>


The last procedure makes use of the inbuilt properties of the logic package. Let us do similar stuff ourselves. 

### - Task 1: (1 Point) Complete the procedure *prefix* that transforms the formula into Prefix notation, i.e., the operator "sits" in front of the two arguments.
Also use different symbols for operators, IMP for implication, AND for conjunction, OR for disjunction and NEG for negation. 

In [85]:
def prefix(s):
    if type(s) is Or:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "OR(" + prefix(child1) + prefix(child2) + ")"
        return string
       ## Fill in the remaining cases
    elif type(s) is Implies:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "IMP(" + prefix(child1) + prefix(child2) + ")"
        return string
    elif type(s) is And:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "AND(" + prefix(child1) + prefix(child2) + ")"
        return string
    elif type(s) is Not:
        child1 = s.children[0]
        string = "NEG(" + prefix(child1) + ")"
        return string
    else:
        return str(s)

Execute the following line of code to check whether your procedure works correctly or not.

In [86]:
for line in content:
    formula = eval(line.strip())
    print(prefix(formula))

IMP(AND(NEG(R)P)Q)
NEG(IMP(IMP(OR(PNEG(Q))R)AND(PR)))
OR(NEG(IMP(PQ))IMP(RP))
OR(IMP(PQ)IMP(QNEG(P)))


If you run your procedure on the input data, you should get something like the following output: 

IMP(AND(NEG(R)P)Q)
etc

### - Task 2: (2 Points) Write a procedure *evaluate* that calculates the value of a formula. 

Input to the procedure is the formula s, and the three truth values for variable P, Q and R. 

This is a procedural way to implement the "meaning" of the operators (which you can read from the Truth Tables)

In [87]:
def evaluate(s, p, q, r):
    if type(s) is Or:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) == True) and (evaluate(child2,p,q,r) == True):
            return True
        elif (evaluate(child1,p,q,r) == True) and (evaluate(child2,p,q,r) == False):
            return True
        elif (evaluate(child1,p,q,r) == False) and (evaluate(child2,p,q,r) == True):
            return True
        else:
            return False
    elif type(s) is Implies:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) == True) and (evaluate(child2,p,q,r) == True):
            return True
        elif (evaluate(child1,p,q,r) == True) and (evaluate(child2,p,q,r) == False):
            return False
        elif (evaluate(child1,p,q,r) == False) and (evaluate(child2,p,q,r) == True):
            return True
        else:
            return True
    elif type(s) is And:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) == True) and (evaluate(child2,p,q,r) == True):
            return True
        elif (evaluate(child1,p,q,r) == True) and (evaluate(child2,p,q,r) == False):
            return False
        elif (evaluate(child1,p,q,r) == False) and (evaluate(child2,p,q,r) == True):
            return False
        else:
            return False
    elif type(s) is Not:
        child1 = s.children[0]
        if (evaluate(child1,p,q,r) == True):
            return False
        else:
            return True     
    elif type(s) is Variable:        
        if str(s) is 'P':
            return p
        elif str(s) is 'S':
            return s
        elif str(s) is 'Q':
            return q
        else:
            return r 
    else:
        print("Something went wrong") 

Execute the following line of code to check whether your procedure works correctly or not.

In [88]:
for l in content:
     print(evaluate(eval(l.strip()),True,True,False))

True
False
True
True


If you run your program on the given input, you should get an answer like this (modulo editing): <br />
True<br />
False<br />
True<br />
True<br />

### - Task 3: (1 Point) Write a procedure *tautology* that calculates whether a given formula (with maximally three variables P,Q and R) is a tautology 
Hint: Use the *evaluate* function you just wrote and implement a "truthtable" approach whether the formula is always evaluated as "true". 

In [89]:
def tautology(s):
    if evaluate(s, True, True, True) and evaluate(s, True, True, False) and evaluate(s, True, False, True) and evaluate(s, True, False, False)\
    and evaluate(s, False, True, True) and evaluate(s, False, True, False) and evaluate(s, False, False, True) and evaluate(s, False, False, False):
        return True
    else:
        return False

In [90]:
for l in content:
     print(repr(tautology(eval(l.strip()))))

False
False
False
True


When you evaluate your code on the provided input, you should get something like the following: <br />
False<br />
False<br />
False<br />
True<br />

## Exercises related to Simple Knowledge Graph Logic 

We will use a very simple, though crude and ugly, representation for Simple Knowledge Graph Logic: Resources are strings, and triples are lists of Resources. So, a triple ( s p o ) will be represented as a list [s,p,o], and a Knowledge Graph with two triples (s1, p1, o1) and (s2, p2, o2) as a list containing two lists: [[s1,p1,o1],[s2,p2,o2]].

Every line contains a knowledge graph (one list). 

Run the following code to load the Knowledge Graph in knowledgegraph.txt

In [91]:
import os 
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'knowledgegraph.txt')
with open(filename) as file:
    content = file.readlines()

Maybe you want to write a function to print this graph for convenience ... (no extra points, though) 

In [92]:
for l in content:
     print(eval(l.strip()))

[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']]
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital']]
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital'], ['Capital', 'a', 'City'], ['Netherlands', 'neighbours', 'Belgium'], ['Netherlands', 'a', 'Country']]


The next two exercises are about syntactic transformation of Knowledge Graphs. The two target formats are simplified versions of syntaxes for the language RDF that we will introduce later. 

### - Task 4a: (1 Point) Write a procedure to transform the Knowledge Graphs into Simple-n-triple syntax. 
Simple-n-triple is a simple (the most simple?) representation format for representing Knowledge Graphs. 

A triple (s,p,o) is simply written as as follows:

     s p o . 
     
As one string, with blanks between s,p and o, and a dot after the triple. 

First, write a function that transforms a triple ['s','p','o'] into a single string "s p o ."

In [93]:
def simplify(triple):
    string = ''
    for i in triple:
        string += i + ' '
    return string + '.'
    


In [94]:
def ntriple(graph):
    line = ''
    for triple in graph:
        line += simplify(triple) + ' '
    return line

Execute the following line of code to check whether your procedure works correctly or not.

In [95]:
for l in content:
     print(ntriple(eval(l.strip())))

s p o . s p1 o2 . s2 p2 o . 
Netherlands name "Netherlands" . Netherlands has_capital Amsterdam . Amsterdam a Capital . 
Netherlands name "Netherlands" . Netherlands has_capital Amsterdam . Amsterdam a Capital . Capital a City . Netherlands neighbours Belgium . Netherlands a Country . 


Your answer could look like this: <br />
['s p o .', 's p1 o2 .', 's2 p2 o .']<br />
['Netherlands name "Netherlands" .', 'Netherlands has_capital Amsterdam .', 'Amsterdam a Capital .']<br />
['Netherlands name "Netherlands" .', 'Netherlands has_capital Amsterdam .', 'Amsterdam a Capital .', 'Capital a City .', 'Netherlands neighbours Belgium .', 'Netherlands a Country .']<br />
but it would also be ok if you return one string, where all string represetations for triples are concatenated into one string. If in doubt, post a question on Piazza. 

### - Task 4b: (1 Point) Write a procedure to transform the Knowledge Graphs into *simpleturtle*. (1 extra point) 
Simpleturtle is a more compact format for representing Knowledge Graphs. 

A triple (s,p,o) is again written like in simple-n-triples: "s p o ." as one string, with blanks between s,p and o, and a dot after the triple. But now, suppose there are more triples with the same "subject", i.e. the first element of the triple, e.g. (s,p1,o1), (s,p2,o2) and (s,p1,o2). These triples will then be written in a more compact format, namely: 

(s p1 o1; p2 o2; p1 o2) 

The ";" is interpreted as disjunction. 

Write such a function simpleturtle:

In [96]:
def simpleturtle(triples):
    new_line = []
    i = 0
    for triple in triples:
        string = ''
        subject = str(triple[0])
        k = 1
        while i < (len(triples) - 1):
            if subject == str(triples[i + 1][0]):
                triples[i + 1][0] = ';'
                duplicate = triples.pop(i + 1)
                triple.extend(duplicate)
                k += 1
            else:
                i += 1

        for item in triple:
            if item == ";":
                string += item + ' '
            elif item == triple[-1]:
                string += item + ' . '
            else:
                string += item + ' '
        
        new_line.append(string)

    return new_line

Check whether your code works correctly: 

In [97]:
for l in content:
     print(simpleturtle(eval(l.strip())))

['s p o ; p1 o2 . ', 's2 p2 o . ']
['Netherlands name "Netherlands" ; has_capital Amsterdam . ', 'Amsterdam a Capital . ']
['Netherlands name "Netherlands" ; has_capital Amsterdam ; neighbours Belgium ; a Country . ', 'Amsterdam a Capital . ', 'Capital a City . ']


Your answer could look like this:<br />
['s p o ; p1 o2 .', 's2 p2 o .']<br />
['Netherlands name "Netherlands ; has_capital Amsterdam .', 'Amsterdam a Capital .']<br />
['Netherlands name "Netherlands" ; has_capital Amsterdam ; neighbours Belgium ; a Country .', 'Amsterdam a Capital .', 'Capital a City .']<br />
but similar representations might also be ok (e.g. as strings). If in doubt, post a question on Piazza.<br />

Most elegant would be if you add indents, such as a representation<br />
[<br />
's p o ; <br />
&nbsp;&nbsp;&nbsp;&nbsp;    p1 o2 .', <br />
's2 p2 o .'<br />
]<br />
This is more close to the "real" Turtle format. 

### - Task 5: (2 Points) Write a procedure to evaluate whether a Triple (s p o) is entailed by your Knowledge Graph

Remember that we can calculate whether a triple is entailed by a Knowledge Graph (w.r.t Simple Knowledge Graph Logic) by simply checking whether the triple is a subgraph of the knowledge graph.  

The triples, on which you can test your implementations are located in the file: entailedTriple.txt. You need to open the file and check for all triples whether it is entailed by the knowledge graphs in knowledgegraph.txt (which is already opened). 

You might want to add auxiliary functions in extra cells if necessary.

In [98]:
def tripleEntail(graph, entailment):
    if entailment in graph:
        return str(graph) + ' entails ' + str(entailment)
    else:
        return str(graph) + ' does NOT entail ' + str(entailment)

Provide some code to check that your function works correctly.  

In [99]:
import os 
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'entailedTriples.txt')
with open(filename) as file:
    entailedTriples = file.readlines()

for l in content:
    for triple in entailedTriples:
        print(tripleEntail(eval(l.strip()), eval(triple.strip())))
    

[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] entails ['s2', 'p2', 'o']
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] does NOT entail ['Netherlands', 'name', '"Netherlands"']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital']] does NOT entail ['s2', 'p2', 'o']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital']] entails ['Netherlands', 'name', '"Netherlands"']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital'], ['Capital', 'a', 'City'], ['Netherlands', 'neighbours', 'Belgium'], ['Netherlands', 'a', 'Country']] does NOT entail ['s2', 'p2', 'o']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital'], ['Capital', 'a', 'City'], ['Netherlands', 'neighbours', 'Belgium'], ['Netherlands', 'a', 'Country']] entails ['

The expected output is a list of relations between graphs and triples<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] entails ['s2','p2','o']<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] does NOT entail ['Netherlands','name','"Netherlands"']<br />
and so on for all graphs and triples. 

### - Task 6: (2 Points) Write a procedure to evaluate whether one Knowledge Graph is entailed by another Knowledge Graph

The graphs, on which you can test your implementations are located in the file: entailedGraphs.txt. 

You might want to add auxiliary functions in extra cells if necessary.

In [100]:
def graphEntail(graph, entailgraph):
    if all(triple in graph for triple in entailgraph):
        return str(graph) + ' entails ' + str(entailgraph)
    else:
        return str(graph) + ' does NOT entail ' + str(entailgraph)

Provide some code to check that your function works correctly.  

In [101]:
import os 
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'entailedGraphs.txt')
with open(filename) as file:
    entailedGraphs = file.readlines()

for l in content:
    for graphs in entailedGraphs:
        print(graphEntail(eval(l.strip()), eval(graphs.strip())))

[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] entails [['s2', 'p2', 'o'], ['s2', 'p2', 'o']]
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] entails [['s', 'p', 'o'], ['s2', 'p2', 'o']]
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] does NOT entail [['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Capital', 'a', 'City'], ['Netherlands', 'neighbours', 'Belgium']]
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] does NOT entail [['Netherlands', 'has_capital', 'Amsterdam'], ['Capital', 'a', 'Country'], ['Netherlands', 'neighbours', 'Belgium']]
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital']] does NOT entail [['s2', 'p2', 'o'], ['s2', 'p2', 'o']]
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'a', 'Capital']] does NOT entail [['s', 'p', 'o'], ['s2', 'p2', 'o']]
[['Netherlands', 'name', '"Nether

The expected output is a list of relations between graphs and graphs<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] entails [['s','p','o'],['s2','p2','o']]<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] entails [['s2','p2','o'],['s2','p2','o']]<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] does NOT entail [['Netherlands','name','"Netherlands"'],['Netherlands','has_capital','Amsterdam'],['Capital','a','City'],['Netherlands','neighbours','Belgium']]
 [['s','p','o'],['s2','p2','o']]<br />
and so on for all graphs and triples. 